<center><h1>Alshehri_Asma_HW4</h1></center>
<br>
<br>

Name: Asma Alshehri
<br>
Github Username: Asma-571
<br>
USC ID: 5168462498

## 1. Time Series Classification Part 1: Feature Creation/Extraction

### (a) Download Data


Package imports

In [1]:
# %pip install bootstrapped

In [2]:
import os
import shutil
import pandas as pd
import glob
import csv
from scipy.stats import trim_mean
import shutil
from sklearn.preprocessing import StandardScaler
import numpy as np
from bootstrapped import bootstrap as bs
from bootstrapped import stats_functions as bs_stats
import shutil
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)


Get the AReM Data Set

Download the AReM data from: https://archive.ics.uci.edu/ml/datasets/
Activity+Recognition+system+based+on+Multisensor+data+fusion+\%28AReM\
%29 . The dataset contains 7 folders that represent seven types of activities. In
each folder, there are multiple files each of which represents an instant of a human
performing an activity.1 Each file containis 6 time series collected from activities
of the same person, which are called avg rss12, var rss12, avg rss13, var rss13,
vg rss23, and ar rss23. There are 88 instances in the dataset, each of which contains 6 time series and each time series has 480 consecutive values.1

Before splitting the data to train and test, In every csv file in AReM I will check for any missing or null values, any rows that contain more or fewer values than expected, which effectively skips over rows with extra commas. Finally checking if a dataset has 480 rows and 7 columns

In [3]:
# time index #Columns: time

In [5]:
dirs = ['bending1/', 'bending2/', 'cycling/', 'lying/', 'sitting/', 'standing/', 'walking/']
for directory in dirs:
    directory_path = f"./AReM/{directory}"
    for filename in os.listdir(directory_path):
        if filename.endswith('.csv'):
            try:
                df = pd.read_csv(os.path.join(directory_path, filename), skiprows=4, error_bad_lines=True)
                if df.shape != (480, 7):
                    print(f"{filename} in {directory_path} does not have 480 rows and 7 columns")
                elif df.isnull().values.any():
                    print(f"{filename} in {directory_path} contains missing or null values")
            except pd.errors.ParserError as e:
                print(f"{filename} in {directory_path} has an error: {e}")


dataset4.csv in ./AReM/bending2/ contains missing or null values
dataset14.csv in ./AReM/cycling/ has an error: Error tokenizing data. C error: Expected 7 fields in line 485, saw 8

dataset9.csv in ./AReM/cycling/ has an error: Error tokenizing data. C error: Expected 7 fields in line 485, saw 8

dataset8.csv in ./AReM/sitting/ does not have 480 rows and 7 columns


There are several issues encountered in multiple datasets in AReM:


_1. Missing or Null Values in'dataset4.csv in ./AReM/bending2/'_


_2. ParserError catched using a try/except in 'dataset14.csv in ./AReM/cycling/'_ 


_3. ParserError catched using a try/except in 'dataset9.csv in ./AReM/cycling/'_


_4. Shapes not aligned in 'dataset8.csv in ./AReM/sitting/'_


The above issues will be resolved by implementing additional code

The values in /AReM/bending2/dataset4.csv are not separated by commas, and are instead clustered together. the following Script will resolve this

In [6]:
df_cl = pd.read_csv('./AReM/bending2/dataset4.csv',skiprows=4)
values_col = df_cl['# Columns: time']
split_cols = values_col.str.split(expand=True)
new_cols = ['col_{}'.format(i) for i in range(7)]
split_cols.columns = new_cols
df_cl = pd.concat([df_cl, split_cols], axis=1)

In [10]:
df_cl.head(5)

,# Columns: time,avg_rss12,var_rss12,avg_rss13,var_rss13,avg_rss23,var_rss23,col_0,col_1,col_2,col_3,col_4,col_5,col_6
0,0 32.50 0.50 0.00 0.00 19.00 1.00,NaN,NaN,NaN,NaN,NaN,NaN,0,32.50,0.50,0.00,0.00,19.00,1.00
1,250 32.50 0.50 0.00 0.00 18.50 0.50,NaN,NaN,NaN,NaN,NaN,NaN,250,32.50,0.50,0.00,0.00,18.50,0.50
2,500 32.75 0.43 1.00 0.00 18.00 0.00,NaN,NaN,NaN,NaN,NaN,NaN,500,32.75,0.43,1.00,0.00,18.00,0.00
3,750 32.50 0.50 0.00 0.00 17.50 0.50,NaN,NaN,NaN,NaN,NaN,NaN,750,32.50,0.50,0.00,0.00,17.50,0.50
4,1000 32.50 0.50 7.50 0.50 17.50 0.87,NaN,NaN,NaN,NaN,NaN,NaN,1000,32.50,0.50,7.50,0.50,17.50,0.87


In [11]:
df_cl= df_cl.drop(['# Columns: time','avg_rss12','var_rss12','avg_rss13','var_rss13','avg_rss23','var_rss23'],axis=1)

In [12]:
df_cl=df_cl.rename(columns={'col_0': '# Columns: time', 'col_1': 'avg_rss12','col_2': 'var_rss12','col_3': 'avg_rss13','col_4': 'var_rss13','col_5': 'avg_rss23','col_6': 'var_rss23'})

In [13]:
df_cl.head(5)

,# Columns: time,avg_rss12,var_rss12,avg_rss13,var_rss13,avg_rss23,var_rss23
0,0,32.50,0.50,0.00,0.00,19.00,1.00
1,250,32.50,0.50,0.00,0.00,18.50,0.50
2,500,32.75,0.43,1.00,0.00,18.00,0.00
3,750,32.50,0.50,0.00,0.00,17.50,0.50
4,1000,32.50,0.50,7.50,0.50,17.50,0.87


In [16]:
# save new df as csv

In [17]:
df_cl.to_csv('./AReM/bending2/dataset4.csv', index=False)

Moving on the issue number 2. The error suggests that the parser has encountered unexpected characters or structures in 'dataset14.csv in ./AReM/cycling/', and 'dataset9.csv in ./AReM/cycling/'. (e.g., missing values, extra commas, ...etc). 

Since I ran missing values test code above, we can narrow the type of error to ' extra commas'



In [19]:
filename = './AReM/cycling/dataset9.csv'

with open(filename, newline='') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        # check if the row has extra commas
        if len(row) > 7:
            print(f"Row {reader.line_num} contains an extra comma: {row}")


Row 485 contains an extra comma: ['119750', '38.33', '0.94', '15.25', '2.17', '20.33', '1.25', '']


In [21]:
filename = './AReM/cycling/dataset14.csv'

with open(filename, newline='') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        # check if the row has extra commas
        if len(row) > 7:
            print(f"Row {reader.line_num} contains an extra comma: {row}")


Row 485 contains an extra comma: ['119750', '34.00', '2.12', '24.50', '1.80', '22.25', '2.59', '']


Extra commas in row 485 are removed manually.

Moving on the the last issue. Apparently in 'dataset8.csv in ./AReM/sitting/', the shape of the dataset doesn’t align With the rest.

In [25]:
df_si = pd.read_csv('./AReM/sitting/dataset8.csv',skiprows=4)
df_si.shape

(479, 7)

After adding one row of time index 13500 between row 50 and 60 and using interpolate()
 [13500,42.00,0.00,17.56,2.69,17.27,3.42]

In [26]:
df_si2 = pd.read_csv('./AReM/sitting/dataset8.csv',skiprows=4)
df_si2.shape

(480, 7)

In [29]:
dirs = ['bending1/', 'bending2/', 'cycling/', 'lying/', 'sitting/', 'standing/', 'walking/']
num_files_processed = 0
num_files_with_issues = 0

for directory in dirs:
    directory_path = f"./AReM/{directory}"
    for filename in os.listdir(directory_path):
        if filename.endswith('.csv'):
            try:
                df = pd.read_csv(os.path.join(directory_path, filename), skiprows=4, error_bad_lines=True)
                if df.shape != (480, 7):
                    print(f"{filename} in {directory_path} does not have 480 rows and 7 columns")
                    num_files_with_issues += 1
                elif df.isnull().values.any():
                    print(f"{filename} in {directory_path} contains missing or null values")
                    num_files_with_issues += 1
                else:
                    num_files_processed += 1
            except pd.errors.ParserError as e:
                print(f"{filename} in {directory_path} has an error: {e}")
                num_files_with_issues += 1

print(f"{num_files_processed} files processed with no issues. {num_files_with_issues} files had issues.")


88 files processed with no issues. 0 files had issues.


In [ ]:
######

In [70]:

# # create an empty DataFrame to hold the combined data
# combined_Or_df = pd.DataFrame()

# # define the directories to loop through
# dirs = ['./AReM/bending1/', './AReM/bending2/', './AReM/cycling/', './AReM/lying/', './AReM/sitting/', './AReM/standing/', './AReM/walking/']

# # loop through the directories
# for d in dirs:
#     # get the path to the directory
#     path = os.path.join(os.getcwd(), d)
    
#     # loop through the CSV files in the directory
#     for filename in os.listdir(path):
#         if filename.endswith(".csv"):
#             # read the CSV file into a DataFrame
#             df = pd.read_csv(os.path.join(path, filename),skiprows=4)
            
#             # add a column to identify which directory the data came from
#             df["activity"] = d.strip("/")
            
#             # append the DataFrame to the combined DataFrame
#             combined_Or_df = combined_Or_df.append(df, ignore_index=True)

# # display the combined DataFrame
# combined_Or_df
# # combined_Or_df = combined_df.drop("activity", axis=1)

In [71]:
# create an empty DataFrame to hold the combined data
comb_df = pd.DataFrame()

# define the directories to loop through
dirs = ['./AReM/bending1/', './AReM/bending2/', './AReM/cycling/', './AReM/lying/', './AReM/sitting/', './AReM/standing/', './AReM/walking/']

# loop through the directories
for d in dirs:
    # get the path to the directory
    path = os.path.join(os.getcwd(), d)
    
    # loop through the CSV files in the directory
    for filename in os.listdir(path):
        if filename.endswith(".csv"):
            # read the CSV file into a DataFrame
            df = pd.read_csv(os.path.join(path, filename),skiprows=4)
            
            # add a column to identify which directory the data came from
            activity = d.strip("/") + '_' + filename.split('.')[0]
            df["activity"] = activity
            
            # append the DataFrame to the combined DataFrame
            comb_df = comb_df.append(df, ignore_index=True)
comb_df=comb_df.drop("# Columns: time", axis=1)

In [72]:
comb_df

,avg_rss12,var_rss12,avg_rss13,var_rss13,avg_rss23,var_rss23,activity
0,42.00,0.00,18.50,0.50,12.00,0.00,./AReM/bending1_dataset7
1,42.00,0.00,18.00,0.00,11.33,0.94,./AReM/bending1_dataset7
2,42.75,0.43,16.75,1.79,18.25,0.43,./AReM/bending1_dataset7
3,42.50,0.50,16.75,0.83,19.00,1.22,./AReM/bending1_dataset7
4,43.00,0.82,16.25,0.83,18.00,0.00,./AReM/bending1_dataset7
...,...,...,...,...,...,...,...
42235,31.50,1.66,12.50,3.20,14.25,4.44,./AReM/walking_dataset9
42236,27.33,1.25,11.33,0.94,20.00,4.00,./AReM/walking_dataset9
42237,37.80,7.68,14.20,2.48,17.25,0.83,./AReM/walking_dataset9
42238,33.75,1.30,15.75,5.21,16.50,2.69,./AReM/walking_dataset9


### (b) Test and Train Data
Keep datasets 1 and 2 in folders bending1 and bending 2, as well as datasets 1,
2, and 3 in other folders as test data and other datasets as train data.

In [31]:
paths_to_directories = ["./AReM_Copy/bending1/", "./AReM_Copy/bending2/"]
for path in paths_to_directories:
    os.makedirs(os.path.join(path, "train_data"), exist_ok=True)
    os.makedirs(os.path.join(path, "test_data"), exist_ok=True)
    path_to_train_data = os.path.join(path, "train_data")
    path_to_test_data = os.path.join(path, "test_data")
    num_datasets = len([filename for filename in os.listdir(path) if filename.startswith("dataset")])
    for i in range(1, num_datasets + 1):
        if i in [1,2]:
            shutil.move(os.path.join(path, f"dataset{i}.csv"), os.path.join(path_to_test_data, f"dataset{i}.csv"))
        else:
            shutil.move(os.path.join(path, f"dataset{i}.csv"), os.path.join(path_to_train_data, f"dataset{i}.csv"))


In [32]:
paths_to_directories = ["./AReM_Copy/cycling/", "./AReM_Copy/lying/","./AReM_Copy/sitting/","./AReM_Copy/standing/","./AReM_Copy/walking/"]
for path in paths_to_directories:
    os.makedirs(os.path.join(path, "train_data"), exist_ok=True)
    os.makedirs(os.path.join(path, "test_data"), exist_ok=True)
    path_to_train_data = os.path.join(path, "train_data")
    path_to_test_data = os.path.join(path, "test_data")
    num_datasets = len([filename for filename in os.listdir(path) if filename.startswith("dataset")])
    for i in range(1, num_datasets + 1):
        if i in [1,2,3]:
            shutil.move(os.path.join(path, f"dataset{i}.csv"), os.path.join(path_to_test_data, f"dataset{i}.csv"))
        else:
            shutil.move(os.path.join(path, f"dataset{i}.csv"), os.path.join(path_to_train_data, f"dataset{i}.csv"))

In [34]:
data_frames_train = []
data_frames_test = []

# Loop over each directory in AReM_Copy
for directory in os.listdir('./AReM_Copy/'):
    if not os.path.isdir(os.path.join('./AReM_Copy', directory)):
        continue
    for subdirectory in os.listdir(os.path.join('./AReM_Copy', directory)):
        if not os.path.isdir(os.path.join('./AReM_Copy', directory, subdirectory)):
            continue
            
        path_to_csv_files = os.path.join('./AReM_Copy', directory, subdirectory)
        
        for csv_file in os.listdir(path_to_csv_files):
            if not csv_file.endswith('.csv'):
                continue
                
            path_to_csv_file = os.path.join(path_to_csv_files, csv_file)
            df = pd.read_csv(path_to_csv_file, skiprows=4)
            if subdirectory == 'train_data':
                data_frames_train.append(df)
            elif subdirectory == 'test_data':
                data_frames_test.append(df)
train_data = pd.concat(data_frames_train, axis=0)
test_data = pd.concat(data_frames_test, axis=0)
combined_df = pd.concat([test_data, train_data], ignore_index=True)
print(len(combined_df))
print(f'test_data = {len(test_data)},traint_data ={len(train_data)}')

42240
test_data = 9120,traint_data =33120


In [42]:
combined_df_no_indx = combined_df.drop('# Columns: time', axis=1)
combined_df_no_indx

,avg_rss12,var_rss12,avg_rss13,var_rss13,avg_rss23,var_rss23
0,39.25,0.43,22.75,0.43,33.75,1.30
1,39.25,0.43,23.00,0.00,33.00,0.00
2,39.25,0.43,23.25,0.43,33.00,0.00
3,39.50,0.50,23.00,0.71,33.00,0.00
4,39.50,0.50,24.00,0.00,33.00,0.00
...,...,...,...,...,...,...
42235,41.75,1.79,11.00,6.00,16.67,2.49
42236,36.33,0.47,16.00,3.16,20.33,1.70
42237,31.50,1.50,21.00,0.00,12.25,7.12
42238,34.25,6.38,12.67,2.49,15.25,4.21


### (c) Feature Extraction

Classification of time series usually needs extracting features from them. In this
problem, we focus on time-domain features.

#### i. Research
Research what types of time-domain features are usually used in time series
classification and list them (examples are minimum, maximum, mean, etc).

_When we want to classify time series data, we need to extract features that describe the shape and patterns in the data. These features can be classified into different categories such as Time-domain features, Frequency Domain Features, and Model-Based Feature._



Here are some common time-domain features that can be computed for a time series dataset:

Mean: The mean of the signal over a given time interval.

Variance: The variance of the signal over a given time interval.

Standard deviation: The standard deviation of the signal over a given time interval.

Minimum value: The minimum value of the signal over a given time interval.

Maximum value: The maximum value of the signal over a given time interval.

Root mean square (RMS) value: The square root of the mean of the squared values of the signal over a given time interval.

Skewness: A measure of the asymmetry of the signal over a given time interval.

Kurtosis: A measure of the "peakedness" of the signal over a given time interval.

Zero crossing rate: The rate at which the signal crosses the zero value over a given time interval.

Auto-correlation: A measure of the similarity between the signal and a delayed version of itself over a given time interval.<font color='blue'>

These features can provide information about the statistical properties, shape, and dynamics of the time series, and can be used to characterize the signal and discriminate between different classes of time series data.
    
<font color='green'> Sources: 
    - https://stats.stackexchange.com/questions/50807/features-for-time-series-classification
    - https://jneuroengrehab.biomedcentral.com/articles/10.1186/1743-0003-7-21

#### ii. Extraction
Extract the time-domain features minimum, maximum, mean, median, standard deviation, first quartile, and third quartile for all of the 6 time series
in each instance. You are free to normalize/standardize features or use them
directly.2

In [103]:

# define the columns to calculate the statistics for
cols = ['avg_rss12', 'var_rss12', 'avg_rss13', 'var_rss13', 'avg_rss23', 'var_rss23']

# group the data based on the activity column and calculate the statistics for each group
stats_df = comb_df[cols + ['activity']].groupby('activity').agg([np.min, np.max, np.mean, np.median, np.std, lambda x: np.percentile(x, 25), lambda x: np.percentile(x, 75)])

# rename the columns to include the respective statistics
new_cols = [col + '_' + stat for col in cols for stat in ['min', 'max', 'mean', 'median', 'std', 'q1', 'q3']]
stats_df.columns = new_cols

# reset the index to make the activity column a regular column
stats_df = stats_df.reset_index()
stats_df

,activity,avg_rss12_min,avg_rss12_max,avg_rss12_mean,avg_rss12_median,avg_rss12_std,avg_rss12_q1,avg_rss12_q3,var_rss12_min,var_rss12_max,...,avg_rss23_std,avg_rss23_q1,avg_rss23_q3,var_rss23_min,var_rss23_max,var_rss23_mean,var_rss23_median,var_rss23_std,var_rss23_q1,var_rss23_q3
0,./AReM/bending1_dataset1,37.25,45.00,40.624792,40.500,1.476967,39.25,42.0000,0.0,1.30,...,2.188449,33.0000,36.00,0.0,1.92,0.570583,0.430,0.582915,0.00,1.3000
1,./AReM/bending1_dataset2,38.00,45.67,42.812812,42.500,1.435550,42.00,43.6700,0.0,1.22,...,1.995255,32.0000,34.50,0.0,3.11,0.571083,0.430,0.601010,0.00,1.3000
2,./AReM/bending1_dataset3,35.00,47.40,43.954500,44.330,1.558835,43.00,45.0000,0.0,1.70,...,1.999604,35.3625,36.50,0.0,1.79,0.493292,0.430,0.513506,0.00,0.9400
3,./AReM/bending1_dataset4,33.00,47.75,42.179813,43.500,3.670666,39.15,45.0000,0.0,3.00,...,3.849448,30.4575,36.33,0.0,2.18,0.613521,0.500,0.524317,0.00,1.0000
4,./AReM/bending1_dataset5,33.00,45.75,41.678063,41.750,2.243490,41.33,42.7500,0.0,2.83,...,2.411026,28.4575,31.25,0.0,1.79,0.383292,0.430,0.389164,0.00,0.5000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,./AReM/walking_dataset5,20.75,46.25,34.763333,35.290,4.742208,31.67,38.2500,0.0,12.68,...,3.174681,14.2500,18.33,0.0,9.39,3.288271,3.270,1.647528,2.05,4.3050
84,./AReM/walking_dataset6,21.50,51.00,34.935812,35.500,4.645944,32.00,38.0625,0.0,12.21,...,3.192058,14.2375,18.25,0.0,10.21,3.280021,3.015,1.700918,2.12,4.5000
85,./AReM/walking_dataset7,18.33,47.67,34.333042,34.750,4.948770,31.25,38.0000,0.0,12.48,...,3.000493,13.7500,18.00,0.0,8.01,3.261583,2.980,1.617290,2.05,4.3200
86,./AReM/walking_dataset8,18.33,45.75,34.599875,35.125,4.731790,31.50,38.0000,0.0,15.37,...,2.905688,14.0000,18.25,0.0,8.86,3.289542,3.015,1.680170,2.12,4.2600


In [96]:
stats_df_noIndex = stats_df.drop('activity', axis=1)

,avg_rss12_min,avg_rss12_max,avg_rss12_mean,avg_rss12_median,avg_rss12_std,avg_rss12_q1,avg_rss12_q3,var_rss12_min,var_rss12_max,var_rss12_mean,...,avg_rss23_std,avg_rss23_q1,avg_rss23_q3,var_rss23_min,var_rss23_max,var_rss23_mean,var_rss23_median,var_rss23_std,var_rss23_q1,var_rss23_q3
0,37.25,45.00,40.624792,40.500,1.476967,39.25,42.0000,0.0,1.30,0.358604,...,2.188449,33.0000,36.00,0.0,1.92,0.570583,0.430,0.582915,0.00,1.3000
1,38.00,45.67,42.812812,42.500,1.435550,42.00,43.6700,0.0,1.22,0.372438,...,1.995255,32.0000,34.50,0.0,3.11,0.571083,0.430,0.601010,0.00,1.3000
2,35.00,47.40,43.954500,44.330,1.558835,43.00,45.0000,0.0,1.70,0.426250,...,1.999604,35.3625,36.50,0.0,1.79,0.493292,0.430,0.513506,0.00,0.9400
3,33.00,47.75,42.179813,43.500,3.670666,39.15,45.0000,0.0,3.00,0.696042,...,3.849448,30.4575,36.33,0.0,2.18,0.613521,0.500,0.524317,0.00,1.0000
4,33.00,45.75,41.678063,41.750,2.243490,41.33,42.7500,0.0,2.83,0.535979,...,2.411026,28.4575,31.25,0.0,1.79,0.383292,0.430,0.389164,0.00,0.5000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,20.75,46.25,34.763333,35.290,4.742208,31.67,38.2500,0.0,12.68,4.223792,...,3.174681,14.2500,18.33,0.0,9.39,3.288271,3.270,1.647528,2.05,4.3050
84,21.50,51.00,34.935812,35.500,4.645944,32.00,38.0625,0.0,12.21,4.115750,...,3.192058,14.2375,18.25,0.0,10.21,3.280021,3.015,1.700918,2.12,4.5000
85,18.33,47.67,34.333042,34.750,4.948770,31.25,38.0000,0.0,12.48,4.396958,...,3.000493,13.7500,18.00,0.0,8.01,3.261583,2.980,1.617290,2.05,4.3200
86,18.33,45.75,34.599875,35.125,4.731790,31.50,38.0000,0.0,15.37,4.398833,...,2.905688,14.0000,18.25,0.0,8.86,3.289542,3.015,1.680170,2.12,4.2600


#### iii. Standard Deviation

In [102]:
# create an empty DataFrame to store results
results_df = pd.DataFrame(columns=['Feature', 'std', 'ci_lower', 'ci_upper'])

# loop through columns and calculate std and confidence intervals
for col in stats_df_noIndex.columns:
    data = stats_df_noIndex[col].to_numpy()
    std = np.std(data)
    std_ci = bs.bootstrap(data, stat_func=bs_stats.std, alpha=0.1)
    results_df = results_df.append({'Feature': col, 'std': std, 
                                    'ci_lower': std_ci.lower_bound, 
                                    'ci_upper': std_ci.upper_bound}, 
                                   ignore_index=True)

results_df

,Feature,std,ci_lower,ci_upper
0,avg_rss12_min,9.515445,8.316092,10.829000
1,avg_rss12_max,4.369322,3.459442,5.431541
2,avg_rss12_mean,5.305300,4.749321,5.919256
3,avg_rss12_median,5.409056,4.854978,6.058866
4,avg_rss12_std,1.762057,1.578340,1.958987
5,avg_rss12_q1,6.118526,5.635953,6.688424
6,avg_rss12_q3,5.109643,4.417823,5.912804
7,var_rss12_min,0.000000,0.000000,0.000000
8,var_rss12_max,5.033882,4.690841,5.468741
9,var_rss12_mean,1.565200,1.436794,1.742884


#### iv. Select Features
iv. Use your judgement to select the three most important time-domain features
(one option may be min, mean, and max).

I suggest using the mean, standard deviation, and either the range (i.e., max-min) or the median. These features are crucial because they provide insight into the spread and skewness of the time-series data.

## 2. ISLR 3.7.4

4. I collect a set of data (n = 100 observations) containing a single
predictor and a quantitative response. I then fit a linear regression
model to the data, as well as a separate cubic regression, i.e. Y =
β0 + β1X + β2X2 + β3X3 + ϵ.


(a) Suppose that the true relationship between X and Y is linear,
i.e. Y = β0 + β1X + ϵ. Consider the training residual sum of
squares (RSS) for the linear regression, and also the training
RSS for the cubic regression. Would we expect one to be lower
than the other, would we expect them to be the same, or is there
not enough information to tell? Justify your answer.


(b) Answer (a) using test rather than training RSS.



(c) Suppose that the true relationship between X and Y is not linear,
but we don’t know how far it is from linear. Consider the training
RSS for the linear regression, and also the training RSS for the
cubic regression. Would we expect one to be lower than the
other, would we expect them to be the same, or is there not
enough information to tell? Justify your answer.

(d) Answer (c) using test rather than training RSS

### (a) Linear Train

If the true relationship between X and Y is linear, then fitting a cubic regression model will likely result in overfitting. Due to the cubic regression's higher degree of flexibility that allows the cubic model to fit more closely to the data points, resulting in a tighter fit. As a result, the cubic regression model will likely have a lower training RSS than the linear regression model, but this does not necessarily mean that it will have a lower test RSS.

### (b) Linear Test

When evaluating a model's performance on a test set, it is likely that the residual sum of squares (RSS) for the cubic regression model will be higher than that of the linear regression model, assuming a linear relationship between the predictor and response variables. This is because the cubic model, with its higher degree of flexibility, may overfit to the training data, resulting in minimal bias reduction. Furthermore, the increased variance of the cubic model in a true linear relationship may offset any bias reduction gained, making the variance reduction provided by a linear model fit more significan


In summary, overfitting leads to a low RSS on the training data and a high RSS on the validation or test data.

### (c) Not Linear Train

The cubic model is expected to exhibit a lower RSS due to its greater degree of flexibility, which enables it to incorporate more degrees of freedom than the linear model.

### (d) Not Linear Testing

It is difficult to determine which model would perform better since we lack information regarding the nature of the data and how non-linear the true relationship between X and Y is.

If the true relationship is only slightly nonlinear, then the linear model may still provide a good fit and have a lower training RSS than the cubic model. But if the true relationship is highly nonlinear, then the cubic model may provide a better fit and have a lower training RSS than the linear model. Therefore, it is important to evaluate the performance of both models using cross-validation or other techniques to ensure that the model selected is the one that provides the best balance between simplicity and accuracy."

In general, a simpler model like linear regression is preferred unless there is evidence that a more complex model like cubic regression is necessary to capture the underlying true relationship.

## 1. Time Series Classification Part 1: Feature Creation/Extraction (HW3 Rerun)